# Toronto Capstone Project

## Segmenting and Clustering Neighborhoods

### Step 1 - Importing the PostcodeData

Import necessary libraries

In [1]:
import pandas as pd
import html5lib
import numpy as np # library to handle data in a vectorized manner

import json # library to handle JSON files

# Install a conda package in the current Jupyter kernel
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

Read and load in the London data

In [2]:
import types
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_3a33d4e155934e6eaa5e22b47b44233e = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='-JjZtQIjKY-QcuS1dbRvsQVLWdVvNg7Q-vJa-aV1aDvq',
    ibm_auth_endpoint="https://iam.eu-gb.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_3a33d4e155934e6eaa5e22b47b44233e.get_object(Bucket='londonberlincapstoneproject-donotdelete-pr-ir0qop9v65xarw',Key='London Postcodes.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

london_df = pd.read_csv(body)

london_df.head()

,pcd,lat,long
0,BR1,51.406271,0.015177
1,BR1,51.406271,0.015177
2,BR1,51.400040,0.016671
3,BR1,51.404517,0.014151
4,BR1,51.408226,0.017578


In [3]:
#london_table = pd.read_csv('London Postcodes.csv')
#london_table
#london_df = pd.DataFrame(london_table)

In [4]:
london_df.count()

pcd     312430
lat     312430
long    312430
dtype: int64

In [5]:
london_df = london_df.groupby(['pcd'], sort = False).mean()
london_df.reset_index(inplace=True)

In [6]:
london_df.head()

,pcd,lat,long
0,BR1,51.410786,0.019448
1,BR2,51.390411,0.021559
2,BR3,51.403565,-0.031345
3,BR4,51.375659,-0.009785
4,BR5,51.389206,0.102520


In [7]:
london_df.count()

pcd     328
lat     328
long    328
dtype: int64

In [58]:
body = client_3a33d4e155934e6eaa5e22b47b44233e.get_object(Bucket='londonberlincapstoneproject-donotdelete-pr-ir0qop9v65xarw',Key='Berlin Postcodes.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

berlin_df = pd.read_csv(body)

In [59]:
#berlin_table = pd.read_csv('London Postcodes.csv')
#berlin_table
#berlin_df = pd.DataFrame(berlin_table)

In [60]:
berlin_df.head()

,Postcode,District
0,10115,Mitte
1,10117,Mitte
2,10119,Mitte
3,10119,Pankow
4,10178,Mitte


In [61]:
berlin_df.count()

Postcode    244
District    244
dtype: int64

In [62]:
berlin_df = berlin_df.groupby(['Postcode'], sort = False).agg(lambda x: ', '.join(x))
berlin_df.reset_index(inplace=True)

In [63]:
berlin_df.head()

,Postcode,District
0,10115,Mitte
1,10117,Mitte
2,10119,"Mitte, Pankow"
3,10178,"Mitte, Friedrichshain-Kreuzberg"
4,10179,"Mitte, Friedrichshain-Kreuzberg"


In [64]:
berlin_df.count()

Postcode    191
District    191
dtype: int64

### Step 2 - Add Latitude & Longitude coordinates into Berlin DataFrame

In [65]:
body = client_3a33d4e155934e6eaa5e22b47b44233e.get_object(Bucket='londonberlincapstoneproject-donotdelete-pr-ir0qop9v65xarw',Key='Berlin Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# If you are reading an Excel file into a pandas DataFrame, replace `read_csv` by `read_excel` in the next statement.
berlin_coord_df = pd.read_csv(body)
berlin_coord_df.head()

,Postcode,Lat,Lng
0,10115,52.53195,13.38380
1,10117,52.51744,13.39632
2,10119,52.53013,13.40551
3,10178,52.52237,13.40921
4,10179,52.51367,13.41519


In [66]:
#berlin_coord_table = pd.read_csv('London Postcodes.csv')
#bercoord_table
#berlin_coord_df = pd.DataFrame(berlin_table)

In [67]:
berlin_df = berlin_df.merge(berlin_coord_df, how='left').reset_index(drop=True)

In [68]:
berlin_df.head()

,Postcode,District,Lat,Lng
0,10115,Mitte,52.53195,13.38380
1,10117,Mitte,52.51744,13.39632
2,10119,"Mitte, Pankow",52.53013,13.40551
3,10178,"Mitte, Friedrichshain-Kreuzberg",52.52237,13.40921
4,10179,"Mitte, Friedrichshain-Kreuzberg",52.51367,13.41519


In [69]:
berlin_df.count()

Postcode    191
District    191
Lat         191
Lng         191
dtype: int64

### Step 3 - Analyse Neighbourhood Data

#### Get Coordinates for London

In [36]:
london = 'London,UK'

geolocator = Nominatim(user_agent="to_explorer")
london_location = geolocator.geocode(london)
london_latitude = london_location.latitude
london_longitude = london_location.longitude
print('The geograpical coordinate of London are {}, {}.'.format(london_latitude, london_longitude))

The geograpical coordinate of London are 51.5073219, -0.1276474.


#### Get Coordinates for Berlin

In [70]:
berlin = 'Berlin, DE'

geolocator = Nominatim(user_agent="to_explorer")
berlin_location = geolocator.geocode(berlin)
berlin_latitude = berlin_location.latitude
berlin_longitude = berlin_location.longitude
print('The geograpical coordinate of Berlin are {}, {}.'.format(berlin_latitude, berlin_longitude))

The geograpical coordinate of Berlin are 52.5170365, 13.3888599.


#### Superimpose Neighbourhoods onto Map

In [73]:
# create map of London using latitude and longitude values
map_london = folium.Map(location=[london_latitude, london_longitude], zoom_start=11)
map_london

In [71]:
# create map of Berlin using latitude and longitude values
map_berlin = folium.Map(location=[berlin_latitude, berlin_longitude], zoom_start=12)
map_berlin

In [76]:
# add markers to map
for lat, lng, pcd, in zip(london_df['lat'], london_df['long'], london_df['pcd']):
    label = '{}'.format(pcd)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

In [72]:
# add markers to map
for lat, lng, pcd, in zip(berlin_df['Lat'], berlin_df['Lng'], berlin_df['Postcode']):
    label = '{}'.format(pcd)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_berlin)  
    
map_berlin

#### Select Neighbourhoods in Downtown Toronto

In [77]:
mitte_data = berlin_df[berlin_df['District'].str.match('Mitte')].reset_index(drop=True)
mitte_data.head()

,Postcode,District,Lat,Lng
0,10115,Mitte,52.53195,13.38380
1,10117,Mitte,52.51744,13.39632
2,10119,"Mitte, Pankow",52.53013,13.40551
3,10178,"Mitte, Friedrichshain-Kreuzberg",52.52237,13.40921
4,10179,"Mitte, Friedrichshain-Kreuzberg",52.51367,13.41519


#### Get Coordinates of Mitte

In [79]:
address = 'Mitte, Berlin'

geolocator = Nominatim(user_agent="to_explorer")
mitte_location = geolocator.geocode(address)
mitte_latitude = mitte_location.latitude
mitte_longitude = location.longitude
print('The geograpical coordinate of Mitte, Berlin  are {}, {}.'.format(mitte_latitude, mitte_longitude))

The geograpical coordinate of Mitte, Berlin  are 52.5176896, 13.3888599.


#### Superimpose Mitte Neighbourhoods onto Map

In [83]:
# create map of Manhattan using latitude and longitude values
map_mitte = folium.Map(location=[mitte_latitude, mitte_longitude], zoom_start=14)

# add markers to map
for lat, lng, label in zip(mitte_data['Lat'], mitte_data['Lng'], mitte_data['Postcode']):
    label = '{}'.format(label)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_mitte)  
    
map_mitte

#### Define Foursquare Credentials and Version

In [ ]:
CLIENT_ID = '*****' # your Foursquare ID
CLIENT_SECRET = '*****' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

<a id='item2'></a>

#### Function to get all Nearby Venues in Downtown Toronto

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Run getNearbyVenues on Downtown Neighbourhoods

In [ ]:
downtown_venues = getNearbyVenues(names=downtown_data['Neighbourhood'],
                                   latitudes=downtown_data['Latitude'],
                                   longitudes=downtown_data['Longitude']
                                  )



#### Check Results

In [ ]:
print(downtown_venues.shape)
downtown_venues.head()

#### Check Number of Returned Venues

In [ ]:
downtown_venues.groupby('Neighbourhood').count()['Venue']

#### Determine How Many Types of Venues Have Been Returned

In [ ]:
print('There are {} uniques categories.'.format(len(downtown_venues['Venue Category'].unique())))

<a id='item3'></a>

#### Analyse Each Neighbourhood

In [ ]:
# one hot encoding
downtown_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_onehot['Neighbourhood'] = downtown_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
downtown_onehot = downtown_onehot[fixed_columns]

downtown_onehot.head()

And let's examine the new dataframe size.

In [ ]:
downtown_onehot.shape

#### Group rows by neighbouhood and by taking the mean of the frequency of occurrence of each category

In [ ]:
downtown_grouped = downtown_onehot.groupby('Neighbourhood').mean().reset_index()
downtown_grouped

#### Confirm the new size

In [ ]:
downtown_grouped.shape

#### Print each neighborhood along with the top 5 most common venues

In [ ]:
num_top_venues = 5

for hood in downtown_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = downtown_grouped[downtown_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

#### Put into a *pandas* dataframe

#### Write a function to sort the venues in descending order.

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Create the new dataframe and display the top 10 venues for each neighborhood.

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = downtown_grouped['Neighbourhood']

for ind in np.arange(downtown_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

<a id='item4'></a>

#### Cluster Neighbourhoods

#### Run *k*-means

In [ ]:
# set number of clusters
kclusters = 5

neighbourhood_grouped_clustering = downtown_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(neighbourhood_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

#### Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [ ]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_merged = downtown_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtown_merged = downtown_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

downtown_merged.head() # check the last columns!

#### Visualise the clusters

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=14)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], downtown_merged['Neighbourhood'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

#### Examine Clusters

#### Cluster 1

In [ ]:
cluster1 = downtown_merged.loc[downtown_merged['Cluster Labels'] == 0, downtown_merged.columns[[2] + list(range(5, downtown_merged.shape[1]))]]
cluster1.head()

#### Cluster 2

In [ ]:
cluster2 = downtown_merged.loc[downtown_merged['Cluster Labels'] == 1, downtown_merged.columns[[2] + list(range(5, downtown_merged.shape[1]))]]
cluster2.head()

#### Cluster 3

In [ ]:
cluster3 = downtown_merged.loc[downtown_merged['Cluster Labels'] == 2, downtown_merged.columns[[2] + list(range(5, downtown_merged.shape[1]))]]
cluster3.head()

#### Cluster 4

In [ ]:
cluster4 = downtown_merged.loc[downtown_merged['Cluster Labels'] == 3, downtown_merged.columns[[2] + list(range(5, downtown_merged.shape[1]))]]
cluster4.head()

#### Cluster 5

In [ ]:
cluster5 = downtown_merged.loc[downtown_merged['Cluster Labels'] == 4, downtown_merged.columns[[2] + list(range(5, downtown_merged.shape[1]))]]
cluster5.head()